In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# 1 Generate the data


In [5]:
from proj1_helpers import *
yb, input_data, ids = load_csv_data("./test.csv")


# 2 Least square methods
## 2.1 Least square with gradient descent 


In [6]:
from costs import *

def least_square(y, tx):
    w = np.pinv(tx) @ y
    loss = costs.
    
    
    

SyntaxError: invalid syntax (<ipython-input-6-ce32ad513326>, line 2)